In [230]:
import keras

In [231]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, GlobalAveragePooling2D, Dense
from keras.models import Model
from keras.optimizers import SGD
import numpy as np
import pickle as pkl
import datetime
from skimage import exposure

# Load Cifar10 dataset

In [232]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

In [233]:
print('The training set has {0} samples with dimentions {1}.'.format(x_train.shape[0], x_train.shape[1:]))
print('The test set has {0} samples with dimentions {1}.'.format(x_test.shape[0], x_test.shape[1:]))

The training set has 50000 samples with dimentions (32, 32, 3).
The test set has 10000 samples with dimentions (32, 32, 3).


In [234]:
NUM_CLASSES = 10
IMG_ROWS, IMG_COLS = 32, 32
BATCH_SIZE = 512
EPOCHS = 20
VAL_SPLIT = .2
LEARNING_RATE = 1e-2
SGD_MOMENTUM = .9

# Preprocess data

In [235]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

In [236]:
x_train -= np.mean(x_train)
x_test -= np.mean(x_test)

In [237]:
x_train /= np.std(x_train)
x_test /= np.std(x_test)

In [238]:
y_train = keras.utils.to_categorical(y_train, NUM_CLASSES)
y_test = keras.utils.to_categorical(y_test, NUM_CLASSES)

In [239]:
validation_split = int((1-VAL_SPLIT)*len(x_train))
x_train, x_val = x_train[:validation_split], x_train[validation_split:]
y_train, y_val = y_train[:validation_split], y_train[validation_split:]

In [240]:
def contrast_stretching(img):
    p2, p98 = np.percentile(img, (2, 98))
    return exposure.rescale_intensity(img, in_range=(p2, p98))

In [241]:
datagen = ImageDataGenerator(preprocessing_function=contrast_stretching)

# Build model

In [242]:
input_tensor = Input(shape=(IMG_ROWS, IMG_COLS, 3))

In [243]:
base_model = VGG16(include_top=False, input_tensor=input_tensor)

In [245]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

In [246]:
vgg16 = Model(inputs=base_model.input, outputs=predictions)

In [247]:
vgg16.compile(optimizer=SGD(lr=LEARNING_RATE, SGD_MOMENTUM), loss='categorical_crossentropy', metrics=['accuracy'])

# Train model

In [248]:
now = datetime.datetime.now()
tensorflow_log_dir = "Graph/" + now.strftime("%Y%m%d-%H%M%S") + "/"

In [249]:
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=tensorflow_log_dir, histogram_freq=0,  
          write_graph=False, write_images=False)

In [ ]:
vgg16.fit_generator(datagen.flow(x_train, y_train, batch_size=BATCH_SIZE),
                    steps_per_epoch=len(x_train) / BATCH_SIZE, 
                    validation_data=ImageDataGenerator().flow(x_val, y_val, batch_size=BATCH_SIZE),
                    validation_steps=len(x_val) / BATCH_SIZE,
                    epochs=EPOCHS, callbacks=[tensorboard_callback], verbose=1)

Epoch 1/100
79/78 [==============================] - 64s 805ms/step - loss: 2.0337 - acc: 0.2939 - val_loss: 1.8914 - val_acc: 0.3465
Epoch 2/100
79/78 [==============================] - 58s 732ms/step - loss: 1.5818 - acc: 0.4458 - val_loss: 1.5521 - val_acc: 0.4496
Epoch 3/100
79/78 [==============================] - 58s 739ms/step - loss: 1.3651 - acc: 0.5245 - val_loss: 1.5957 - val_acc: 0.4761
Epoch 4/100
79/78 [==============================] - 58s 734ms/step - loss: 1.2352 - acc: 0.5736 - val_loss: 1.7893 - val_acc: 0.4006
Epoch 5/100
79/78 [==============================] - 58s 736ms/step - loss: 1.1374 - acc: 0.6068 - val_loss: 1.0898 - val_acc: 0.6206
Epoch 6/100
79/78 [==============================] - 58s 739ms/step - loss: 1.0583 - acc: 0.6319 - val_loss: 1.2888 - val_acc: 0.5730
Epoch 7/100
79/78 [==============================] - 58s 731ms/step - loss: 0.9999 - acc: 0.6516 - val_loss: 1.0442 - val_acc: 0.6389
Epoch 8/100
79/78 [==============================] - 58s 730ms

In [212]:
vgg16.evaluate(x_test, y_test, verbose=1)

10000/10000 [==============================] - 6s 552us/step


[0.7941863952636719, 0.7319]